In [15]:
import numpy 
import sklearn
import pandas as pd
import xgboost as xgb


In [16]:
Data_full = pd.read_csv("train.csv", index_col = "PassengerId")
features = ["Age", "CryoSleep", "VIP", 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', "HomePlanet","Destination", "Cabin"]

y = Data_full.Transported
X = Data_full.drop("Transported", axis=1)[features]

X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y)


# print(X_train[X_train["Cabin"].isna()])


cleaning

In [17]:
# ship side
X_train["portside"] = X_train["Cabin"].str[-1]
X_train["portside"] = X_train["portside"].map({"p":True, "S":False, })

X_valid["portside"] = X_valid["Cabin"].str[-1]
X_valid["portside"] = X_valid["portside"].map({"p":True, "S":False})


# # deck 
# X_train["portside"] = X_train["Cabin"].str[1]
# X_train["portside"] = X_train["portside"].map({"p":True, "S":False})

# X_valid["portside"] = X_valid["Cabin"].str[1]
# X_valid["portside"] = X_valid["portside"].map({"p":True, "S":False})

X_train.drop("Cabin", inplace=True, axis=1)
X_valid.drop("Cabin", inplace=True, axis=1)



In [18]:
# pipelines 
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', sklearn.preprocessing.StandardScaler()),
    ('model', xgb.XGBClassifier())
])

In [19]:
from sklearn.impute import SimpleImputer

imputer_mode = SimpleImputer(strategy='most_frequent')
imputer_costant = SimpleImputer(strategy='constant', fill_value=0)
imputer_mean = imputer_spending = SimpleImputer(strategy='mean')


# fill na with 0
fill_NA_with_0 = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"] 
X_train[fill_NA_with_0] = imputer_costant.fit_transform(X_train[fill_NA_with_0])
X_valid[fill_NA_with_0] = imputer_costant.transform(X_valid[fill_NA_with_0])

# fill mode: boolena values 
bool_inputer_cols = ["CryoSleep", "VIP", "Destination", "HomePlanet"]
X_train[bool_inputer_cols] = imputer_mode.fit_transform(X_train[bool_inputer_cols])
X_valid[bool_inputer_cols] = imputer_mode.transform(X_valid[bool_inputer_cols])

# fill adv: age
adv_inputer_cols = ["Age"]
X_train[adv_inputer_cols] = imputer_mean.fit_transform(X_train[adv_inputer_cols])
X_valid[adv_inputer_cols] = imputer_mean.transform(X_valid[adv_inputer_cols])


# for col in X_train.columns:
    # print(X_train[col].value_counts())


# for col in X_train.columns:
#     print(col, X_train[col].isna().sum())

# print()
# for i in Data_full.columns:
#     print(i)




In [ ]:
# one hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()


# # one hot encode Home Planet
train_encoded = encoder.fit_transform(X_train[['HomePlanet']])
encoded_df_train = pd.DataFrame(train_encoded.toarray(), columns=encoder.get_feature_names_out(['HomePlanet']))
encoded_df_train.index = X_train.index
X_train = pd.concat([X_train, encoded_df_train], axis=1)
X_train.drop("HomePlanet", axis=1, inplace=True)

val_encoded = encoder.transform(X_valid[['HomePlanet']])                 # ask chat how to put keep column names
encoded_df_val = pd.DataFrame(val_encoded.toarray(), columns=encoder.get_feature_names_out(['HomePlanet']))
encoded_df_val.index = X_valid.index
X_valid = pd.concat([X_valid, encoded_df_val], axis=1)
X_valid.drop("HomePlanet", axis=1, inplace=True)


# one hot encode destination
train_encoded = encoder.fit_transform(X_train[['Destination']])
encoded_df_train = pd.DataFrame(train_encoded.toarray(), columns=encoder.get_feature_names_out(['Destination']))
encoded_df_train.index = X_train.index
X_train = pd.concat([X_train, encoded_df_train], axis=1)
X_train.drop("Destination", axis=1, inplace=True)

val_encoded = encoder.transform(X_valid[['Destination']])                 # ask chat how to put keep column names
encoded_df_val = pd.DataFrame(val_encoded.toarray(), columns=encoder.get_feature_names_out(['Destination']))
encoded_df_val.index = X_valid.index
X_valid = pd.concat([X_valid, encoded_df_val], axis=1)
X_valid.drop("Destination", axis=1, inplace=True)

X_train


,Age,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,portside,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
PassengerId,,,,,,,,,,,,,,,
6490_03,27.0,False,False,0.0,6924.0,849.0,149.0,1166.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0
6431_03,27.0,False,False,30.0,0.0,763.0,0.0,57.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0
0890_01,16.0,False,False,172.0,0.0,338.0,9.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,1.0
8923_01,19.0,False,False,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0
6000_02,41.0,False,False,1.0,0.0,808.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0875_01,38.0,False,False,0.0,0.0,0.0,0.0,0.0,False,1.0,0.0,0.0,0.0,0.0,1.0
1304_03,10.0,False,False,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0
4981_03,24.0,False,False,530.0,0.0,0.0,0.0,179.0,False,1.0,0.0,0.0,0.0,0.0,1.0


In [22]:
# train model
from sklearn.pipeline import Pipeline

Model = Pipeline([
    ('scaler', sklearn.preprocessing.StandardScaler()),
    ('model', xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42,
        use_label_encoder=False))
])

Model.fit(X_train, y_train)
score = Model.score(X_valid, y_valid)
print(score)
# print("MAE:")
# print(sklearn.metrics.mean_absolute_error(y_valid, preds) * 100)

# runs 
# 1 : 31.4543
# 2: 28.408
# 3: 27.7449
# 4: 27.9839



0.7999080036798528


/Users/tuckernichols/Documents/GitHub/SpaceShipTitanic_MachineLearning/.venv/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [20:17:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
